**Aplikacja Flask**

Import bibliotek

In [3]:
import numpy as np
import pandas as pd
import pickle
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import requests
from tensorflow import keras
from collections import Counter
import tensorflow as tf
from keras.layers import LSTM, Bidirectional, Dense, Dropout, Activation, Embedding
from keras.utils import np_utils
from tensorflow.keras.models import Sequential, load_model
from keras.optimizers import Adamax
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
import random
import re
from nltk import flatten


Stworzenie funkcji do generowania tekstu

In [4]:
def get_lyrics(model, start, no_words, temp):

  start_string_list =  [start]

  input = tf.expand_dims([dictionary_enumerated[s] for s in start_string_list], 0)

  song=start

  model.reset_states()
  for i in range(no_words):
      predicted = (tf.squeeze(model(input), 0))/ temp
      predicted_id = tf.random.categorical(predicted, num_samples=1)[-1,0].numpy()
      input = tf.expand_dims([predicted_id], 0)
      song=song+' ' + enumerated_dictionary[predicted_id]

  return song

Funkcja do pobrania danych

In [5]:
def get_data(genre):
  Words=[]
  df=pd.read_excel(f'dataset/df_{genre}.xlsx')
  for l in df['lyrics']:
    Words.append(re.split(r' ', l))
  Words=flatten(Words)
  Words=[x for x in Words if x]

  dictionary = sorted(set(Words))

  return dictionary

Budowa aplikacji

In [14]:

#!ngrok authtoken 

app = Flask(__name__, template_folder='template', static_folder='static')
run_with_ngrok(app)
genres=['pop', 'rock', 'jazz', 'rap', 'country', 'blues']

@app.route('/')
def home():
  return render_template('index.html', genres=genres)

@app.route('/predict',methods=['POST'])
def predict():
  genre = request.form.get('genres')

  #zebranie danych
  dictionary=get_data(genre)
  dictionary_enumerated = {u: i for i, u in enumerate(dictionary)}
  enumerated_dictionary = np.array(dictionary)

  model = pickle.load(open(f'models/model_{genre}.pkl', 'rb'))

  # #wybranie losowego słowa początkowego z listy
  start_words=['i', 'you', 'she', 'he', 'we']
  start=random.choice(start_words)

  # #wygenerowanie piosenki
  generated=get_lyrics(model, start, 10, 0.1)
  return render_template('index.html',song=f'Wygenerowana piosenka w stylu {genre}: {generated}', genres=genres)

ngrok_tunnel = ngrok.connect(5000)
if __name__ == '__main__':
    app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c866-35-185-255-207.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Jun/2023 15:09:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jun/2023 15:09:12] "GET /static/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jun/2023 15:09:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Jun/2023 15:09:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jun/2023 15:09:16] "GET /static/main.css HTTP/1.1" 304 -
